# 50.007 Machine Learning
## Group Project

## Part 1
Report the precision, recall and F scores of such a baseline system for each dataset:
- EN dataset
  - Entity scores:
    - Entity  precision: 0.5348
    - Entity  recall: 0.7656
    - Entity  F: 0.6297
  - Sentiment scores:
    - Sentiment  precision: 0.3902
    - Sentiment  recall: 0.5586
    - Sentiment  F: 0.4595
- FR dataset
  - Entity scores:
    - Entity  precision: 0.1670
    - Entity  recall: 0.7815
    - Entity  F: 0.2751
  - Sentiment scores:
    - Sentiment  precision: 0.0709
    - Sentiment  recall: 0.3319
    - Sentiment  F: 0.1169

In [ ]:
# import statements
import numpy as np
import math
import copy

# global variables
emission_dict = {} # emission_dict[x][y] gives e(x|y)
tags_list = []
transition_dict = {}
pi_dict = {}
decoding_list = []

In [ ]:
# function that takes in the filename for the training data
# returns the training data as a list in the form: [ [x_1, y_1], [x_2, y_2], ...]
def read_training_data(training_filename):
    training_file = open(training_filename, "r", encoding="utf-8")
    
    word_sequences = []
    tag_sequences = []
    
    current_word_sequence = []
    current_tag_sequence = []
    
    for line in training_file:
        training_word_and_tag = line.strip().split(" ")
        
        # add the current word and tag to the current word sequence and current tag sequence
        if (len(training_word_and_tag) == 2):
            current_word_sequence += [training_word_and_tag[0]]
            current_tag_sequence += [training_word_and_tag[1]]
        
        # when reaching a new sentence (empty line), add the previous word sequence and tag sequence to the lists of
        # word sequences and tag sequences respectively. Assume the training data ends with an empty line
        else:
            word_sequences += [copy.deepcopy(current_word_sequence)]
            tag_sequences += [copy.deepcopy(current_tag_sequence)]
            
            current_word_sequence = []
            current_tag_sequence = []
        
    training_file.close()

    return word_sequences, tag_sequences

# function that takes in the filename of the training data and optional k value
# creates/ returns the global variable "emission_dict". emission_dict[x][y] gives the value e(x|y)
def create_emission_dict(training_filename, k=1):
    # global variables
    global emission_dict
    global tags_list

    # count_tags_dict[y] gives the total number of words tagged as y
    count_tags_dict = {} 

    # count_x_tagged_as_y_dict[x_i][y_j] gives the number of times each observed variable x_i
    # was tagged as state y_j in the training data
    count_x_tagged_as_y_dict = {}

    # read training data
    word_sequences, tags_sequences = read_training_data(training_filename)

    # fill up count_tags_dict and count_x_tagged_as_y_dict
    for sequence_index in range(0, training_data)
    for training_point in training_data:
        x = training_point[0]
        y = training_point[1]

        # account for creating dictionary entry for the first time
        if not(y in count_tags_dict.keys()):
            count_tags_dict[y] = 0

        count_tags_dict[y] += 1

        # account for creating dictionary entry for the first time
        if not(x in count_x_tagged_as_y_dict.keys()): 
            count_x_tagged_as_y_dict[x] = {}
        if not(y in count_x_tagged_as_y_dict[x].keys()):
            count_x_tagged_as_y_dict[x][y] = 0

        count_x_tagged_as_y_dict[x][y] += 1
        
    tags_list = count_tags_dict.keys()

    # fill up emission_dict
    for training_point in training_data:
        x = training_point[0]
        y = training_point[1]

        # account for creating dictionary entry for the first time
        if not(x in emission_dict.keys()):
            emission_dict[x] = {}

        emission_dict[x][y] = count_x_tagged_as_y_dict[x][y] / (count_tags_dict[y] + k)

    # add entry for #UNK#
    emission_dict["#UNK#"] = {}
    for tag in tags_list: # iterate over all the tags used in training
        emission_dict["#UNK#"][tag] = k / (count_tags_dict[tag] + k)

    return emission_dict

# function that takes in observed variable x and hidden state y
# returns emission parameter e(x|y)
def emission(x, y, training_filename, k=1):
    # global variables
    global emission_dict
    global tags_list

    # account for invalid tag
    if not(y in tags_list):
        # print(f"Invalid tag: {y}")
        return 0

    # account for case where word was not in training data
    if not(x in emission_dict.keys()): # treat x as "#UNK#"
        result = emission_dict["#UNK#"][y] # result = k / (count_tags_dict[y] + k)
    else: # if word is was in training data
        # account for x never tagged as y before during traininng
        if not(y in emission_dict[x].keys()):
            emission_dict[x][y] = 0

        result = emission_dict[x][y] 

    return result

# function that takes in the filename for the test data
# returns the test data as a list in the form: [x1, x2, ...]
def read_test_data(test_filename):
    test_file = open(test_filename, "r", encoding="utf-8")
    
    test_data = []

    for line in test_file:
        test_point = line.strip()
        test_data += [test_point]

    test_file.close()

    return test_data

# function that takes in a filename and a list of results in the form: [ [x1, tag1], [x2, tag2], ...]
# writes the results to a file specified by the filename
def write_result(result_filename, results):
    result_file = open(result_filename, "w" ,encoding="utf-8")
    
    for result in results:
        # account for empty lines
        if (len(result) == 0):
            result_file.write("\n")
        else:
            result_file.write(result[0] + " " + result[1] + "\n")

    result_file.close()

# function that takes in the filenames for the training data and test data
# produces the tag y* = arg_max_y e(x|y) for each word in the test data
# writes the results to a file specified by the filename
# returns the results as a list in the form: [ [x1, y*1], [x2, y*2], ... ]
def simple_sentiment_analysis(training_filename, test_filename, result_filename, k=1):
    global emission_dict
    global tags_list

    # reset global variables
    emission_dict = {}
    tags_list = []

    emission_dict = {}
    create_emission_dict(training_filename, k)
    test_data = read_test_data(test_filename)
    results = []

    for test_variable in test_data:
        # account for empty lines
        if (len(test_variable) == 0):
            results += [""]
        else: # find the tag that gives the highest value for e(test_variable | tag)
            predicted_tag = ""
            highest_emission_value = 0

            for tag in tags_list:
                current_emission_value = emission(test_variable, tag, training_filename)

                if current_emission_value > highest_emission_value:
                    highest_emission_value = current_emission_value
                    predicted_tag = tag

            results += [[test_variable, predicted_tag]]

    write_result(result_filename, results)

    return results

In [ ]:
# # test case
# # create training data for test case
# test_case_train_file = open("p1_test_train", "w")
# test_case_train_file.write("word1 tag1\n")
# test_case_train_file.write("word1 tag1\n")
# test_case_train_file.write("word1 tag1\n")
# test_case_train_file.write("word1 tag2\n")
# test_case_train_file.write("\n")
# test_case_train_file.write("word2 tag2\n")
# test_case_train_file.write("word2 tag2\n")
# test_case_train_file.write("word2 tag2\n")
# test_case_train_file.write("\n")
# test_case_train_file.write("word3 tag3")
# test_case_train_file.close()

# # create test data for test case
# test_case_test_file = open("p1_test_in", "w")
# test_case_test_file.write("word1\n")
# test_case_test_file.write("word2\n")
# test_case_test_file.write("word3\n")
# test_case_test_file.write("unknown_word")
# test_case_test_file.close()

# # create expected output for test case
# test_case_expected_file = open("p1_test_out", "w")
# test_case_expected_file.write("word1 tag1\n")
# test_case_expected_file.write("word2 tag2\n")
# test_case_expected_file.write("word3 tag3\n")
# test_case_expected_file.write("unknown_word tag3")
# test_case_expected_file.close()

# # perform the test
# test_case_prediction = simple_sentiment_analysis("p1_test_train", "p1_test_in", "p1_test_prediction")
# test_case_expected = read_training_data("p1_test_out")

# # show results for the test
# print("\nTest case emission_dict:")
# print(emission_dict)
# print("")

# for i in range(len(test_case_prediction)):
#     test_case_passed = True

#     if test_case_prediction[i][1] != test_case_expected[i][1]:
#         print("Test case failed.")
#         print(f"Word: {test_case_prediction[i][0]}")
#         print(f"Tag: {test_case_prediction[i][1]}")
#         print(f"Expected tag: {test_case_expected[i][1]}\n")

#         test_case_passed = False

# print(f"Test case passed: {test_case_passed}")

## Part 2

In [ ]:
# global variables
emission_dict = {} # emission_dict[x][y] gives e(x|y)
tags_list = []
transition_dict = {}
pi_dict = {}

def create_transition_dict(training_filename):
    
    global transition_dict
    training_set = read_training_data(training_filename)
    
    count_yi_minus_1_yi = {}
    count_yi_minus_1 = {}
    
    # Count occurrences of yi-1 and yi
    # ===============================================
    for i in range(len(training_set)):
        if i == 0:
            yi_minus_1 = 'START'
        else:
            yi_minus_1 = training_set[i - 1][1]
        yi = training_set[i][1]
        
        # Create new entry for count(y-1) if it doesnt exist yet
        if yi_minus_1 not in count_yi_minus_1:
            count_yi_minus_1[yi_minus_1] = 0
            
        # Add +1 for the y-1 entry in the count_yi_minus_1 dict    
        count_yi_minus_1[yi_minus_1] += 1
        
        # Create new entry for count(y-1,yi) if it doesnt exist yet
        if (yi_minus_1, yi) not in count_yi_minus_1_yi:
            count_yi_minus_1_yi[(yi_minus_1, yi)] = 0
        
        # Add +1 for the y-1 entry in the (count_yi_minus_1, yi) dict 
        count_yi_minus_1_yi[(yi_minus_1, yi)] += 1

        
    # Calculate transition and put it inside transition_dict
    # ===============================================
    for (yi_minus_1, yi), count in count_yi_minus_1_yi.items():
        transition_dict[(yi_minus_1, yi)] = count / count_yi_minus_1[yi_minus_1]

        
    # Calculate q(STOP|yn) and q(y1|START)
    # ===============================================
    y_n = training_set[-1][1]
    
    # Add entry in dicts for 'STOP'
    transition_dict[(y_n, 'STOP')] = 0
    count_yi_minus_1['STOP'] = 0
    
    # Basically 1 / Count(y_n)
    # +1 for count_yi_minus_1[y_n] + 1 as it does not contain the last y_n
    transition_dict[(y_n, 'STOP')] = 1 / (count_yi_minus_1[y_n] + 1)
    
    # Technically always 1 since there's only 1 start and one combi of (y0, START)
    transition_dict[('START', training_set[0][1])] = 1 / (count_yi_minus_1['START'])

    return transition_dict

# function that takes in observed variable x and hidden state y
# returns transition parameter q(yi|yi-1)
def transition(yi_minus_1, yi):
    # global variables
    global transition_dict

    if (yi_minus_1, yi) not in transition_dict.keys():
        transition_dict[(yi_minus_1, yi)] = 0

    result = transition_dict[(yi_minus_1, yi)]

    return result
    
# function that takes in the filenames for the training data and test data
# creates the table of pi values
def viterby_first_order(training_filename, test_filename, k=1):
    # global variables
    global emission_dict
    global transition_dict
    global pi_dict
    global tags_list

    tags_list_w_start_stop = list(tags_list)
    
    emission_dict = create_emission_dict(training_filename, k)
    create_transition_dict(training_filename)
    
    # training data as a list in the form: [ [x_1, y_1], [x_2, y_2], ...]
    test_data = read_training_data(test_filename)
    print("LENGTH TR DATA:", len(test_data))
    
    
    # initialization
    if ("START" not in tags_list_w_start_stop):
        tags_list_w_start_stop += ["START"]
    
    
    if ("STOP" not in tags_list_w_start_stop):
        tags_list_w_start_stop += ["STOP"]
    
    for x in range(0, len(test_data)+1):
        for v in tags_list_w_start_stop:
            if x not in pi_dict.keys():
                pi_dict[x] = {}
                
            pi_dict[x][v] = 0
        
    pi_dict[0]["START"] = 1
    
    
    # for each observed variable
    for j in range(0, len(test_data)):
        
        x_j_plus_1 = test_data[j][0] # refers to the jth word (to calculate emission)
        
        # for each hidden state v
        for v in tags_list_w_start_stop:
            
            # pi(j+1, v) = max over all u { pi(j,u) * transition(u, v) * emissision(x_j_plus_1, v) }
            max_pi_val = float('-inf')
            
            for u in tags_list_w_start_stop:

                pi = pi_dict[j][u]  
                trans = transition(u, v)
                emi = emission(x_j_plus_1, v, training_filename)
                
                if trans != 0:
                    trans = math.log(trans)
                if emi != 0:
                    emi = math.log(emi)               
                if pi > 0:
                    pi = math.log(pi)
                
                current_pi_val = pi + trans + emi

                # save the value that maximises
                if (current_pi_val > max_pi_val):
                    max_pi_val = current_pi_val
        
            pi_dict[j+1][v] = max_pi_val

    # print("pii dict: ", pi_dict)
    # print("trasition: ",transition(u, v))
    # print("emission: ",emission(x_j_plus_1, v, training_filename))
            
    # final step
    max_pi_val = float('-inf')
    
    # for each hidden state u
    for u in tags_list_w_start_stop:
        pi = pi_dict[len(test_data)][u]
        trans = transition(u, "STOP")
        
        if trans != 0:
            trans = math.log(trans)
        if pi > 0:
            pi = math.log(pi)

        current_pi_val = pi + trans
        
        # save the value that maximises
        if (current_pi_val > max_pi_val):
            max_pi_val = current_pi_val

#     if max_pi_val == 0:
#         max_pi_val = float('-inf')
        
    pi_dict[len(test_data)]["STOP"]  = max_pi_val

    return pi_dict

# print(transition_dict)
# print(emission_dict)

In [ ]:
def write_result_viterby(test_filename, result_filename, results):
    
    test_data = read_test_data(test_filename)
    
    with open(result_filename, "w" ,encoding="utf-8") as fp:
        
        for word,tag in zip(test_data, results):
            # account for empty lines
            if(len(word) == 0):
                fp.write("\n")
            else:
                fp.write(word[0] + " " + tag + "\n")
    fp.close()

def write_result(result_filename, results):
    result_file = open(result_filename, "w" ,encoding="utf-8")
    
    for result in results:
        # account for empty lines
        if (len(result) == 0):
            result_file.write("\n")
        else:
            result_file.write(result[0] + " " + result[1] + "\n")

    result_file.close()            

In [ ]:
def viterby_backtracking(test_filename, result_filename):
    global emission_dict
    global transition_dict
    global pi_dict
    global tags_list
    global decoding_list

    tags_list_w_start_stop = list(tags_list)
    
    # check final layer argmax
    argmax = float('-inf')
    currentmax = 0
    argmax_index = 0
    
    for u in tags_list_w_start_stop:
        pi = pi_dict[len(pi_dict)-1][u]
        trans = transition(u, "STOP")
        
        if trans != 0:
            trans = math.log(trans)
        
        if pi == 0:
            pi = float('-inf')
        
        currentmax = pi + trans
        
        if currentmax > argmax:
            argmax = currentmax
            argmax_index = u
        
    decoding_list.append(argmax_index)
    
    
    # Backtrack rest of pi_dict
    for j in range(len(pi_dict)-2, 0, -1):
        
        argmax = float('-inf')
        currentmax = 1
        argmax_index = 0
    
        for u in tags_list_w_start_stop:
    
            pi = pi_dict[j][u]
            trans = transition(u, decoding_list[-1])
            
            if trans != 0:
                trans = math.log(trans)
            if pi == 0:
                pi = float('-inf')

            currentmax = pi + trans

            if currentmax > argmax:
                argmax = currentmax
                argmax_index = u
        
        decoding_list.append(argmax_index)
        
    decoding_list = decoding_list[::-1]
    
    write_result_viterby(test_filename, result_filename, decoding_list)
            
    return decoding_list   

In [ ]:
# global variables
emission_dict = {} # emission_dict[x][y] gives e(x|y)
tags_list = []
transition_dict = {}
pi_dict = {}
decoding_list = []

In [ ]:
# perform prediction for the EN dataset
en_results = simple_sentiment_analysis("EN/train", "EN/dev.in", "EN/dev.p1.out")

# evaluate prediction for the EN dataset
!python "evalResult.py" "EN/dev.out" "EN/dev.p1.out"

In [ ]:
viterby_first_order("EN/train", "EN/dev.in")
viterby_backtracking("EN/dev.in", "EN/dev.p2.out")

In [ ]:
!python "evalResult.py" "EN/dev.out" "EN/dev.p2.out"

===================================================================

In [ ]:
# global variables
emission_dict = {} # emission_dict[x][y] gives e(x|y)
tags_list = []
transition_dict = {}
pi_dict = {}
decoding_list = []

In [ ]:
# # perform prediction for the FR dataset
fr_results = simple_sentiment_analysis("FR/train", "FR/dev.in", "FR/dev.p1.out")

# # evaluate prediction for the FR dataset
!python "evalResult.py" "FR/dev.out" "FR/dev.p1.out"

In [ ]:
viterby_first_order("FR/train", "FR/dev.in")
viterby_backtracking("FR/dev.in", "FR/dev.p2.out")

In [ ]:
!python "evalResult.py" "FR/dev.out" "FR/dev.p2.out"